In [1]:
import sys
sys.path.append("../")

In [2]:
import lib.utils as utils
import lib.jpeg as jpg
import numpy as np
from tqdm import tqdm
import os, glob

In [3]:
#psnr, ssim and nrmse
valid_custom_metrics = utils.CustomMetric()

In [4]:
#Loading the training dataset
root_folder = "../../dataset/dataset_frames/cropped/valid"

dataset_x = []
dataset_y = []

for file_ in tqdm(glob.iglob(root_folder+"/*.jpg")):
       
        img = utils.open_image(file_)
        #print(file_)
        if img is None:
            print("Corrupted dataset!")

        qtable_luma_50, qtable_chroma_50 = jpg.generate_qtables(quality_factor=50)
        qtable_luma_45, qtable_chroma_45 = jpg.generate_qtables(quality_factor=45)
        qtable_luma_40, qtable_chroma_40 = jpg.generate_qtables(quality_factor=40)
        qtable_luma_35, qtable_chroma_35 = jpg.generate_qtables(quality_factor=35)
        qtable_luma_30, qtable_chroma_30 = jpg.generate_qtables(quality_factor=30)
        qtable_luma_25, qtable_chroma_25 = jpg.generate_qtables(quality_factor=25)
        qtable_luma_20, qtable_chroma_20 = jpg.generate_qtables(quality_factor=20)
        qtable_luma_15, qtable_chroma_15 = jpg.generate_qtables(quality_factor=15)
        qtable_luma_10, qtable_chroma_10 = jpg.generate_qtables(quality_factor=10)

        dct_q10 = jpg.encode_image(img, qtable_luma_10, qtable_chroma_10)
        dct_q15 = jpg.encode_image(img, qtable_luma_15, qtable_chroma_15)
        dct_q20 = jpg.encode_image(img, qtable_luma_20, qtable_chroma_20)
        dct_q25 = jpg.encode_image(img, qtable_luma_25, qtable_chroma_25)
        dct_q30 = jpg.encode_image(img, qtable_luma_30, qtable_chroma_30)
        dct_q35 = jpg.encode_image(img, qtable_luma_35, qtable_chroma_35)
        dct_q40 = jpg.encode_image(img, qtable_luma_40, qtable_chroma_40)
        dct_q45 = jpg.encode_image(img, qtable_luma_45, qtable_chroma_45)
        dct_q50 = jpg.encode_image(img, qtable_luma_50, qtable_chroma_50)
        
        img_q10 = jpg.decode_image(dct_q10, qtable_luma_10, qtable_chroma_10)
        img_q15 = jpg.decode_image(dct_q15, qtable_luma_15, qtable_chroma_15)
        img_q20 = jpg.decode_image(dct_q20, qtable_luma_20, qtable_chroma_20)
        img_q25 = jpg.decode_image(dct_q25, qtable_luma_25, qtable_chroma_25)
        img_q30 = jpg.decode_image(dct_q30, qtable_luma_30, qtable_chroma_30)
        img_q35 = jpg.decode_image(dct_q35, qtable_luma_35, qtable_chroma_35)
        img_q40 = jpg.decode_image(dct_q40, qtable_luma_40, qtable_chroma_40)
        img_q45 = jpg.decode_image(dct_q45, qtable_luma_45, qtable_chroma_45)
        img_q50 = jpg.decode_image(dct_q50, qtable_luma_50, qtable_chroma_50)
        
        dataset_x.append((img_q10, img_q15, img_q20, img_q25, img_q30, img_q35, img_q40, img_q45))
        dataset_y.append(img_q50)

      
        

60it [05:59,  5.99s/it]


In [5]:
#psnr, ssim and nrmse
metric_q10 = utils.CustomMetric()
metric_q15 = utils.CustomMetric()
metric_q20 = utils.CustomMetric()
metric_q25 = utils.CustomMetric()
metric_q30 = utils.CustomMetric()
metric_q35 = utils.CustomMetric()
metric_q40 = utils.CustomMetric()
metric_q45 = utils.CustomMetric()

In [6]:
dataset_size = len(dataset_x)

for index in tqdm(range(0, dataset_size)):

    batch_y = dataset_y[index]
    (batch_x_q10, batch_x_q15, batch_x_q20, batch_x_q25, batch_x_q30, batch_x_q35, batch_x_q40, batch_x_q45) = dataset_x[index]
    metric_q10.feed(np.expand_dims(batch_y, axis=0), np.expand_dims(batch_x_q10, axis=0))
    metric_q15.feed(np.expand_dims(batch_y, axis=0), np.expand_dims(batch_x_q15, axis=0))    
    metric_q20.feed(np.expand_dims(batch_y, axis=0), np.expand_dims(batch_x_q20, axis=0))
    metric_q25.feed(np.expand_dims(batch_y, axis=0), np.expand_dims(batch_x_q25, axis=0))
    metric_q30.feed(np.expand_dims(batch_y, axis=0), np.expand_dims(batch_x_q30, axis=0))
    metric_q35.feed(np.expand_dims(batch_y, axis=0), np.expand_dims(batch_x_q35, axis=0))
    metric_q40.feed(np.expand_dims(batch_y, axis=0), np.expand_dims(batch_x_q40, axis=0))
    metric_q45.feed(np.expand_dims(batch_y, axis=0), np.expand_dims(batch_x_q45, axis=0))

  0%|                                                                                                                        | 0/60 [00:00<?, ?it/s]..\lib\utils.py:30: UserWarning: DEPRECATED: skimage.measure.compare_psnr has been moved to skimage.metrics.peak_signal_noise_ratio. It will be removed from skimage.measure in version 0.18.
  compare_psnr(batch_y[index], predictions[index], data_range=255)), axis=None)
..\lib\utils.py:32: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  compare_ssim(batch_y[index], predictions[index], multichannel=True)), axis=None)
..\lib\utils.py:34: UserWarning: DEPRECATED: skimage.measure.compare_nrmse has been moved to skimage.metrics.normalized_root_mse. It will be removed from skimage.measure in version 0.18.
  compare_nrmse(batch_y[index], predictions[index])), axis=None)
100%|██████████████████████████████████████████████████████

In [7]:
print(len(metric_q10.buffer_psnr), len(metric_q15.buffer_psnr), len(metric_q20.buffer_psnr))



60 60 60


In [8]:
psnr, ssim, nrmse = metric_q10.result()
print("Q10", psnr, ssim, nrmse)

psnr, ssim, nrmse = metric_q15.result()
print("Q15", psnr, ssim, nrmse)

psnr, ssim, nrmse = metric_q20.result()
print("Q20", psnr, ssim, nrmse)

psnr, ssim, nrmse = metric_q25.result()
print("Q25", psnr, ssim, nrmse)

psnr, ssim, nrmse = metric_q30.result()
print("Q30", psnr, ssim, nrmse)

psnr, ssim, nrmse = metric_q35.result()
print("Q35", psnr, ssim, nrmse)

psnr, ssim, nrmse = metric_q40.result()
print("Q40", psnr, ssim, nrmse)

psnr, ssim, nrmse = metric_q45.result()
print("Q45", psnr, ssim, nrmse)

Q10 31.170199142242396 0.7941824439909608 0.18881403985464223
Q15 33.07406200587216 0.8378350002586764 0.1538688819171812
Q20 34.09661262771748 0.8526766018790339 0.1455573822258887
Q25 35.34429295284896 0.9025881442189537 0.11179805397464987
Q30 36.15535297536487 0.9029873763420769 0.11863921091207873
Q35 37.08916402107546 0.9150631638062996 0.1087519219792388
Q40 38.65229471605657 0.9427018650991692 0.08087183449414917
Q45 40.30165297716656 0.9523841494460104 0.07143843476053699
